In [3]:
import pandas as pd
from apify_client import ApifyClient
from datetime import datetime


apify_client = ApifyClient('apify_api_IY3hoxDRTo2d2iF7NoZXPIOtFlDb8J1cSkPQ')

In [4]:
# Start an actor and wait for it to finish
position_df = pd.DataFrame()
job_titles = ["Data Analyst", "Data Engineer"]#, "Data Scientist", "Software Developer", "Product Manager", "Digital Marketer"]
locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancourver"} 
# locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancourver",
#              "w+CAIQICIITW9udHJlYWw=": "Montreal", "w+CAIQICIHQ2FsZ2FyeQ==": "Calgary", "w+CAIQICIIRWRtb250b24=": "Edmonton"}  # UULE

for uule in locations:
    print(locations[uule])
    for jt in job_titles:
        print(jt)
        run_input = {
            "csvFriendlyOutput": True,
            "includeUnfilteredResults": False,
            "maxConcurrency": 10,
            "maxPagesPerQuery": 3,
            "queries": f"https://www.google.com/search?ibp=htl;jobs&q={jt}&uule={uule}",
            "saveHtml": False,
            "saveHtmlToKeyValueStore": False,
        }

        actor_call = apify_client.actor(
            'dan.scraper/google-jobs-scraper').call(run_input=run_input)

        dataset_items = apify_client.dataset(
            actor_call['defaultDatasetId']).list_items().items

        d = pd.DataFrame(dataset_items)
        d["query"] = jt
        d["location"] = locations[uule]
        d["run_time"] = str(datetime.now())

        position_df = pd.concat([position_df, d])

    print("="*30)
    break # Only Run For Toronto

Toronto
Data Analyst
Data Engineer


In [5]:
position_df.groupby("location")["query"].value_counts()

location  query        
Toronto   Data Engineer    30
Name: count, dtype: int64

In [19]:
# _ = position_df.pop("thumbnail") # not useful, with na values
position_df.to_csv("../data/raw_google_1129.csv", index=False)


In [7]:
position_df.head()

,query,location,run_time,title,companyName,via,description,jobHighlights,relatedLinks,extras,metadata,applyLink
0,Data Engineer,Toronto,2024-02-25 01:43:11.042136,Junior Data Engineer,Scotiabank,via Scotiabank Jobs,Requisition ID: 193348\n\nJoin a purpose drive...,[{'items': ['Requisition ID: 193348 Join a pu...,"[{'link': 'http://www.scotiabank.com/', 'text'...","[2 days ago, Full-time, No degree mentioned]","{'postedAt': '2 days ago', 'scheduleType': 'Fu...",{'title': '.nFg2eb{font-weight:500}.Bi6Ddc{fon...
1,Data Engineer,Toronto,2024-02-25 01:43:11.042136,Sr. Data Engineer,EY,via EY Careers,"At EY, you’ll have the chance to build a caree...","[{'items': ['At EY, you’ll have the chance to ...","[{'link': 'http://www.ey.com/', 'text': 'ey.co...","[1 day ago, Full-time]","{'postedAt': '1 day ago', 'scheduleType': 'Ful...","{'title': 'Apply on EY Careers', 'link': 'http..."
2,Data Engineer,Toronto,2024-02-25 01:43:11.042136,Senior Data Engineer,AdventInfotech,via LinkedIn,Senior Data Engineer:\n\nWe at Advent Infotech...,[{'items': ['Senior Data Engineer: We at Adve...,[{'link': 'https://www.google.com/search?sca_e...,"[2 days ago, Full-time]","{'postedAt': '2 days ago', 'scheduleType': 'Fu...","{'title': 'Apply on LinkedIn', 'link': 'https:..."
3,Data Engineer,Toronto,2024-02-25 01:43:11.042136,"Sr Data Engineer, Data Engineering",Cardinal Health,via Cardinal Health,"Cardinal Health Canada, with over 1300 employe...","[{'items': ['Cardinal Health Canada, with over...","[{'link': 'http://www.cardinal.com/', 'text': ...",[Full-time and Contractor],{'scheduleType': 'Full-time and Contractor'},"{'title': 'Apply on Cardinal Health', 'link': ..."
4,Data Engineer,Toronto,2024-02-25 01:43:11.042136,Data Engineer (AWS),Iris Software Inc.,via LinkedIn,"Iris's direct client, one of the leading Banki...","[{'items': ['Iris's direct client, one of the ...","[{'link': 'http://www.irissoftware.com/', 'tex...","[4 days ago, Full-time and Temp work, No degre...","{'postedAt': '4 days ago', 'scheduleType': 'Fu...","{'title': 'Apply on LinkedIn', 'link': 'https:..."


In [9]:
from opensearchpy import OpenSearch, helpers

In [10]:
host = 'search-swift-hire-dev-jfmldmym4cfbiwdhwmtuqq6ihy.us-west-2.es.amazonaws.com'
port = 443
auth = ('swift', 'Hire123!') # For testing only. Don't store credentials in code.

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [11]:
index_name = "swift_dev"

if not client.indices.exists(index_name):
    client.indices.create(index=index_name)

def doc_generator(df):
    for i, row in df.iterrows():
        doc = {
            "_index": index_name,
            "_source": row.to_dict(),
        }
        yield doc

helpers.bulk(client, doc_generator(position_df))

print("Data Saved to ES")

Data Saved to ES


In [12]:
position_df["query"].value_counts()

query
Data Engineer    30
Name: count, dtype: int64